# Multimodal Transformer Model - MOSEI

In this notebook, we implement the MulT Model (Multimodal Transformer for Unaligned Multimodal Language Sequences), and test it on CMU-MOSEI dataset.

## Importing Libraries

In [1]:
import os
import sys
import torch

import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

## Importing Helper Functions

In [2]:
from helpers.data.get_data import get_dataloader
from helpers.unimodals.common_models import Identity, MLP
from helpers.fusions.mult import MULTModel
from helpers.fusions.common_fusions import Concat
from helpers.training.supervised_learning import train, test
from helpers.utils.common_utils import get_default_device, save_results

## Loading the Dataset

In [3]:
traindata, validdata, testdata = get_dataloader('data/mosei_senti_data.pkl', robust_test=False, max_pad=True)

In [5]:
device = get_default_device()
print(device)

cuda


## Model Architecture

In [6]:
class HParams():
    num_heads = 8
    layers = 4
    attn_dropout = 0.1
    attn_dropout_modalities = [0, 0, 0.1]
    relu_dropout = 0.1
    res_dropout = 0.1
    out_dropout = 0.1
    embed_dropout = 0.2
    embed_dim = 40
    attn_mask = True
    output_dim = 1
    all_steps = False

In [7]:
encoders = [
    Identity().to(device), 
    Identity().to(device), 
    Identity().to(device),
    
]

fusion = MULTModel(3, [35, 74, 300], hyp_params=HParams).to(device)

head = Identity().to(device)

## Training

In [8]:
FILE_NAME = 'mosei_mult_r0'

In [ ]:
train_losses, valid_losses = train(
    encoders, fusion, head, traindata, validdata, 30, 
    task="regression", optimtype=torch.optim.AdamW, early_stop=False, is_packed=False, 
    lr=1e-3, clip_val=1.0, save=f'models/{FILE_NAME}.pt', 
    weight_decay=0.01, objective=torch.nn.L1Loss())

Epoch [1/30]
Train Loss: 0.7832
Valid Loss: 0.6622
Saving Best

Epoch [2/30]
Train Loss: 0.6829
Valid Loss: 0.6662

Epoch [3/30]
Train Loss: 0.6525
Valid Loss: 0.6240
Saving Best

Epoch [4/30]


In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.title('Loss vs. No. of epochs')

In [ ]:
save_results(train_losses, valid_losses, f'results/{FILE_NAME}.pkl')

## Testing

In [ ]:
model = torch.load(f'models/{FILE_NAME}.pt').to(device)

test(model, test_dataloaders_all=testdata, dataset='mosi', is_packed=False,
     criterion=torch.nn.L1Loss(), task='posneg-classification', no_robust=True)